# A/B-тестирование рекомендаций в корзине. Доставка суши <a class=tocSkip>
**Продукт** - сайт и приложение доставки суши.  

**Цель** - Переработать рекомендации в корзине и проверить с помощью A/B-теста - значимы ли изменения.
    
**Коротко о внедрении**:       
- Сейчас в рекомендациях в корзине лежат товары из категории роллы, которые с 99% вероятностью уже есть в корзине в том или ином виде на момент просмотра рекомендаций.
- Логично и то, что человек на стадии корзины находится уже с необходимым количеством еды, чтобы насытиться. Предлагая основное блюдо, нам тяжело будет допродать. Разве что гость удалит один из товаров и заменит его рекомендованным.   
- Вместо этого планируется добавить товары, которых скорее всего в корзине нет и они хорошо дополнят уже собранную корзину: напитки и десерты. Это соответственно и разница контента в группе А и Б.

**Задачи**:
1. Спланировать A/B-тест:
- Определить метрики, сплит и метод раздачи меток,
- Определить размер выборки и False positive rate на имеющихся данных.

2. Провести A/B-тест и сделать анализ, 
3. Написать вывод и рекомендации.

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Планирование-A/B-теста" data-toc-modified-id="Планирование-A/B-теста-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Планирование A/B-теста</a></span><ul class="toc-item"><li><span><a href="#Определяем-метрики,-сплит-и-метод-раздачи-меток" data-toc-modified-id="Определяем-метрики,-сплит-и-метод-раздачи-меток-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Определяем метрики, сплит и метод раздачи меток</a></span></li><li><span><a href="#Определяем-размер-выборки-и-False-positive-rate" data-toc-modified-id="Определяем-размер-выборки-и-False-positive-rate-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Определяем размер выборки и False positive rate</a></span></li></ul></li><li><span><a href="#Проведение-A/B-теста-и-анализ" data-toc-modified-id="Проведение-A/B-теста-и-анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Проведение A/B-теста и анализ</a></span><ul class="toc-item"><li><span><a href="#Оценка-изменения-аплифта-метрик-во-времени" data-toc-modified-id="Оценка-изменения-аплифта-метрик-во-времени-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Оценка изменения аплифта метрик во времени</a></span></li><li><span><a href="#Расчет-статистической-значимости" data-toc-modified-id="Расчет-статистической-значимости-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Расчет статистической значимости</a></span></li><li><span><a href="#Изучение-изменения-p-value-во-времени" data-toc-modified-id="Изучение-изменения-p-value-во-времени-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Изучение изменения p-value во времени</a></span></li></ul></li><li><span><a href="#Выводы-и-рекомендации" data-toc-modified-id="Выводы-и-рекомендации-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы и рекомендации</a></span></li></ul></div>

## Планирование A/B-теста

### Определяем метрики, сплит и метод раздачи меток

В этом тесте мы оценим следующие метрики:
- конверсионность рекомендаций,
- дополнительно оценим средний чек в двух группах.

Конверсионность рекомендаций мы будем оценивать по заказам: 
- Заказ отправлен с добавленным товаром из рекомендации - конверсия произошла. 
- Заказ без товара из рекомендаций - конверсия не произошла.   

Основной выбрана именно эта метрика, так как в ней мы ожидаем максимальный аплифт, что в свою очередь поможет сократить длительность эксперимента и четко интерпретировать результаты. 

---

Другие параметры:  
**Сплит**: 50/50 - наименьшая длительность теста.  
**Метод раздачи меток** - ленивая раздача пользователям, которые оказались в корзине. Так нам проще всего будет соблюсти баланс групп.       
**Метод проверки**: t-тест. Подходящий для нас инструмент для оценки среднего. Данные будут представлены в виде 0 и 1 и не будут иметь выбросов. Для среднего чека также подойдет.

### Определяем размер выборки и False positive rate 

Для того, чтобы определить размер выборки и длительность эксперимента нам потребуется посчитать базовую конверсию и прикинуть аплифт.  
Дополнительно проверим собираемые данные на адекватность, посчитать False positive rate.

Начнем с определения размера выборки.

In [1]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import stats

# зададим константы базовых покупок и покупок с добавлением рекомендаций
PURCHASE = 5098
PUR_ADD_REC = 311

In [2]:
# посчитаем базовую конверсию
print('Базовая конверсия рекомендаций:', round((PUR_ADD_REC / PURCHASE), 3))

Базовая конверсия рекомендаций: 0.061


Посчитали размер выборки на калькуляторе с заданными параметрами: 
- Мощность и уровень значимости: 80% и 5%,
- relative MDE: 35%  
Получили размер выборки: 2,065.

Да, ожидаемый MDE стоит весьма высокий, но и эффективность рекомендаций должна сильно вырасти. Также доводы за небольшую выборку:
- Отслеживаем заказы, что является весьма стабильной метрикой. В отличии от просмотров, в которых может быть много выбросов, случайных событий и т.д.
- Обладаем не очень большим трафиком по заказам, чтобы растягивать тест сильнее.

Далее посчитаем False positive rate для тестовой выборки без изменений. Применим t-тест 1000 раз, разбивая наши данные на группы а и б случайным образом.

In [3]:
# приведем данные к набору 0 и 1 и объединим в один df
a_a_test = pd.concat([pd.Series(0, index=np.arange(PURCHASE - PUR_ADD_REC))
                      , pd.Series(1, index=np.arange(PUR_ADD_REC))], ignore_index=True).to_frame()
a_a_test.columns = ['conversion']

# Сделаем тысячу раз по 2 выборки и посчитаем для них стат значимость
N = 1000
p_vals = []

for i in tqdm(range(N)):
    a_a_test['group'] = np.random.choice(['a', 'b'], len(a_a_test), p=[1/2, 1/2])
    
    p_val = stats.ttest_ind(
        a_a_test[a_a_test['group']=='a'].conversion,
        a_a_test[a_a_test['group']=='b'].conversion,
        equal_var = False
    ).pvalue
    
    if p_val < 0.05:
        p_vals.append(p_val)
        
print('False positive rate:', round((len(p_vals) / N), 4))

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 220.30it/s]

False positive rate: 0.047


Получили FPR = 4,7%, что говорит об адекватности наших данных при стат значимости 5%.

## Проведение A/B-теста и анализ

Для проведение теста нам необходимо собрать 2 выборки А и B по 2065 заказов и оценить в них конверсию. 
Дополнительно оценим средний чек по двум группам. 

В процессе проведение - контролируем сплит.  

**<<< To be continued...**

### Оценка изменения аплифта метрик во времени

### Расчет статистической значимости

### Изучение изменения p-value во времени

## Выводы и рекомендации